In [2]:
# loading data from NSRDB (https://nsrdb.nrel.gov/data-viewer)

import pandas as pd
import numpy as np

In [5]:
# elevation angle is 90 - (solar zenith angle)
# estimate global tilted irradiance by:
# GTI = direct normal irradiance (DNI) * sin(elevation + panel tilt) + diffuse horizontal irradiance (DHI)

# panels are rated according to peak solar hours (PSH), which corresponds to a solar irradiance of 1000 W/m^2
# if we compute GTI, we can estimate the percentage of this peak output that a panel will produce (minus 14% system losses)

df = pd.read_csv('caltech_solar_radiation.csv', parse_dates=True)

df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']])
df.drop(['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
df.set_index('datetime', inplace=True)

df['Solar Elevation Angle'] = 90 - df['Solar Zenith Angle']

PANEL_TILT = 34.5 # degrees

df['GTI'] = df['DNI'] * np.sin(np.radians(df['Solar Elevation Angle'])) + df['DHI']

display(df)

,DHI,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure,GHI,Solar Zenith Angle,Solar Elevation Angle,GTI
datetime,,,,,,,,,,,,
2019-01-01 00:30:00,0,0,-6.0,0.14,4.0,49.72,3.5,972,0,166.60,-76.60,0.0
2019-01-01 01:30:00,0,0,-6.0,0.14,3.9,51.16,3.1,973,0,156.62,-66.62,0.0
2019-01-01 02:30:00,0,0,-6.2,0.14,4.2,50.79,3.0,973,0,144.63,-54.63,0.0
2019-01-01 03:30:00,0,0,-6.9,0.14,4.4,48.52,2.9,973,0,132.25,-42.25,0.0
2019-01-01 04:30:00,0,0,-7.0,0.14,4.3,48.74,2.7,974,0,119.90,-29.90,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:30:00,0,0,2.5,0.13,2.9,62.78,9.2,969,0,121.52,-31.52,0.0
2021-12-31 20:30:00,0,0,2.4,0.13,3.6,64.60,8.7,969,0,133.88,-43.88,0.0
2021-12-31 21:30:00,0,0,2.1,0.13,3.9,65.10,8.3,969,0,146.24,-56.24,0.0


In [6]:
# so a 1kW system will produce 1kWh of DC for 1 PSH (idealized)

# inverter efficiency is around 95%

# system losses are around 14%

# set the system size (recall that we assume 80amp AC charger (19 kW), so we need roughly 25 kW of DC solar capacity in the best case)
dc_system_size = 25.0 # kW

INVERTER_EFFICIENCY = 0.95
SYSTEM_LOSS = 0.14

df['Solar Generation (kWh)'] = dc_system_size * (df['GTI'] / 1000) * (1-SYSTEM_LOSS) * INVERTER_EFFICIENCY

display(df)

,DHI,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure,GHI,Solar Zenith Angle,Solar Elevation Angle,GTI,Solar Generation (kWh)
datetime,,,,,,,,,,,,,
2019-01-01 00:30:00,0,0,-6.0,0.14,4.0,49.72,3.5,972,0,166.60,-76.60,0.0,0.0
2019-01-01 01:30:00,0,0,-6.0,0.14,3.9,51.16,3.1,973,0,156.62,-66.62,0.0,0.0
2019-01-01 02:30:00,0,0,-6.2,0.14,4.2,50.79,3.0,973,0,144.63,-54.63,0.0,0.0
2019-01-01 03:30:00,0,0,-6.9,0.14,4.4,48.52,2.9,973,0,132.25,-42.25,0.0,0.0
2019-01-01 04:30:00,0,0,-7.0,0.14,4.3,48.74,2.7,974,0,119.90,-29.90,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 19:30:00,0,0,2.5,0.13,2.9,62.78,9.2,969,0,121.52,-31.52,0.0,0.0
2021-12-31 20:30:00,0,0,2.4,0.13,3.6,64.60,8.7,969,0,133.88,-43.88,0.0,0.0
2021-12-31 21:30:00,0,0,2.1,0.13,3.9,65.10,8.3,969,0,146.24,-56.24,0.0,0.0


In [7]:
print(df['Solar Generation (kWh)'].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3989112295302455, 5.134524148132914, 8.555027369109983, 11.019249323619881, 12.26020143188816, 12.193587125952716, 10.74480774094836, 8.110126673370207, 4.618189372228092, 0.9680063657132002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.3456411411593059, 4.971656954185295, 8.282537332146376, 10.78281174503666, 12.0193983113239, 11.927273034013444, 10.486185809887681, 7.903071051362298, 4.45253569800997, 0.9185949798370666, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.352907948628224, 5.042415042658971, 8.427646335125004, 10.888172314392794, 12.143516189957204, 12.071931454247377, 10.68756938407597, 8.111535964341764, 4.6099312052619466, 1.0142063211610663, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2627604163976538, 4.818470192816757, 8.157246422317476, 10.755858121073004, 12.007888359147637, 11.955352545940421, 10.533253511352308, 7.947359240906926, 4.55525177605915